<a href="https://colab.research.google.com/github/cnboonhan/colab/blob/main/mixtral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# run this if in google colab environment to install dependencies
! pip install -q condacolab
import condacolab
condacolab.install()
! conda install llama-index qdrant_client torch transformers pyTelegramBotAPI

import subprocess
subprocess.Popen(["ollama", "serve"])
! ollama pull mistral

In [2]:
from pathlib import Path
import qdrant_client
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    download_loader,
)
from llama_index.llms import Ollama
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.readers import SimpleDirectoryReader

llm = Ollama(model="mistral")

JSONReader = download_loader("SimpleDirectoryReader")
loader = SimpleDirectoryReader(input_dir=Path('./data'))
documents = loader.load_data()
client = qdrant_client.QdrantClient(
    path="./qdrant_data"
)
vector_store = QdrantVectorStore(client=client, collection_name="data")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm,embed_model="local")
index = VectorStoreIndex.from_documents(documents,service_context=service_context,storage_context=storage_context)
query_engine = index.as_query_engine()

/home/cnboonhan/miniconda3/envs/rag_faq_bot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 684/684 [00:00<00:00, 2.83MB/s]
model.safetensors: 100%|██████████| 133M/133M [00:01<00:00, 106MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 2.68MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 80.8MB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 62.2MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 636kB/s]


In [8]:
print(query_engine.query("What does apigw.yaml do? Give expected resources to be created and what configuration options there are").response)

 The provided file, `apigw.yaml`, is a CloudFormation template that defines parameters, metadata, conditions, and resources for creating an Amazon API Gateway (apigw) infrastructure. It sets up various configurations such as the environment, description, naming convention, custom domain name, VPC endpoint ID, and more.

Expected resources to be created:
1. Amazon ApiGateway RestApi
2. Custom domain name (if provided)
3. API path mappings (if a custom domain name is used)
4. VPC link for BFF microservice (if provided)
5. Lambda authorizer (if configured with a name and ARN)

Configuration options:
1. Environment - The environment in which to provision the infrastructure resources
2. Description - A description of this Api Gateway
3. ApiName - A naming convention for the API
4. ApiGatewayStageName - A stage name for the API gateway
5. CustomDomainName - A domain name for this API gateway (optional)
6. CustomDomainApiMappingBasePath - Path mapping for the custom domain name (optional)
7. 

In [10]:
print(query_engine.query("Which of the provided yaml files will deploy a red hat SSO server?").response)

 Based on the provided YAML file, the `microservice-rhsso.yaml` file appears to contain configurations for deploying a Red Hat Single Sign-On (SSO) server using AWS services such as Elastic Load Balancer (NLB), Target Group, and VPCLink. Therefore, this file is responsible for deploying a Red Hat SSO server in the specified environment according to the provided configurations.


In [11]:
import telebot
key = input("Provide Telegram Bot Key")
bot = telebot.TeleBot(key, parse_mode=None)
@bot.message_handler(func=lambda m: True)
def echo_all(message):
  response = query_engine.query(message.text)
  bot.reply_to(message, response)
bot.infinity_polling()

2024-01-23 06:49:57,799 (__init__.py:1027 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2024-01-23 06:49:57,800 (__init__.py:1029 MainThread) ERROR - TeleBot: "Break infinity polling"
